# setup

In [2]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [3]:
datafile='data7.parquet'
testfile='servers7.parquet'
serverfile='test7.parquet'


In [4]:

gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [5]:
pd_data=pd.read_parquet(datafile)
pd_test=pd.read_parquet(testfile)
pd_servers=pd.read_parquet(serverfile)


In [15]:
pd_servers['proto']

index
781742    udp
781743    tcp
781744    tcp
781745    tcp
781746    tcp
         ... 
684757    tcp
684758    tcp
684759    tcp
684760    tcp
684761    tcp
Name: proto, Length: 1030719, dtype: object

In [6]:

#Just the UDP flows
udpFnormal=pd_data.loc[pd_data['proto']=='udp']
udpFtest=pd_test.loc[pd_test['proto']=='udp']
udpFservers=pd_servers.loc[pd_servers['proto']=='udp']

#Just the TCP flows
tcpFnormal=pd_data.loc[pd_data['proto']=='tcp']
tcpFtest=pd_test.loc[pd_test['proto']=='tcp']
tcpFservers=pd_servers.loc[pd_servers['proto']=='tcp']

In [7]:
upNormal = tcpFnormal.groupby(['src_ip'])['up_bytes'].sum()
downNormal = tcpFnormal.groupby(['src_ip'])['down_bytes'].sum()

ratioNormal=pd.DataFrame(upNormal/downNormal,columns=['ratio'])
upTest = tcpFtest.groupby(['src_ip'])['up_bytes'].sum()
downTest = tcpFtest.groupby(['src_ip'])['down_bytes'].sum()
ratioTest=pd.DataFrame(upTest/downTest,columns=['ratio'])


In [8]:
normal443 = tcpFnormal.loc[tcpFnormal['port']==443]
normaldns = udpFnormal.loc[udpFnormal['port']==53]
normaldnst = tcpFnormal.loc[tcpFnormal['port']==53]

normal443.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709948,1232265,192.168.107.196,157.240.212.35,tcp,443,13896,113280
709949,1232398,192.168.107.196,157.240.212.35,tcp,443,45276,419472
709950,1232531,192.168.107.196,157.240.212.35,tcp,443,20618,174565
709951,1232562,192.168.107.196,157.240.212.35,tcp,443,4648,30893
709952,1232681,192.168.107.196,157.240.212.35,tcp,443,10239,90989


In [9]:
normaldns.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
710000,1248724,192.168.107.196,192.168.107.227,udp,53,242,559
710001,1248730,192.168.107.196,192.168.107.227,udp,53,220,545


In [10]:

normaldnst.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,


In [14]:
mean443 =tcpFnormal.loc[tcpFnormal['port']==443]['up_bytes'].sum()/tcpFnormal.loc[tcpFnormal['port']==443]['down_bytes'] .sum()
mean53 =udpFnormal.loc[udpFnormal['port']==53]['up_bytes'].sum()/udpFnormal.loc[udpFnormal['port']==53]['down_bytes'] .sum()

upN443 = tcpFnormal.loc[tcpFnormal['port']==443].groupby(['src_ip'])['up_bytes'].sum()
upN53 = udpFnormal.loc[udpFnormal['port']==53].groupby(['src_ip'])['up_bytes'].sum()

downN443 = tcpFnormal.loc[tcpFnormal['port']==443].groupby(['src_ip'])['down_bytes'].sum()
downN53 = udpFnormal.loc[udpFnormal['port']==53].groupby(['src_ip'])['down_bytes'].sum()


In [13]:
upN443test = tcpFtest.loc[tcpFtest['port']==443].groupby(['src_ip'])['up_bytes'].sum()
upN53test = udpFtest.loc[udpFtest['port']==53].groupby(['src_ip'])['up_bytes'].sum()

downN443test = tcpFtest.loc[tcpFtest['port']==443].groupby(['src_ip'])['down_bytes'].sum()
downN53test = udpFtest.loc[udpFtest['port']==53].groupby(['src_ip'])['down_bytes'].sum()


In [15]:
downN53test

Series([], Name: down_bytes, dtype: int64)

In [16]:
ratioNormal443=pd.DataFrame(upN443/downN443,columns=['ratio'])
ratioNormal53=pd.DataFrame(upN53/downN53,columns=['ratio'])

ratioTest443=pd.DataFrame(upN443test/downN443test,columns=['ratio'])
ratioTest53=pd.DataFrame(upN53test/downN53test,columns=['ratio'])


In [203]:
mean443*1.10

0.11911159134613934

In [222]:
ax443 = ratioTest443.plot(rot='vertical')

 mod 10

SyntaxError: invalid syntax (1290547328.py, line 3)

In [205]:
# ver os que estao acima de mean443 * 1.1

d = pd.DataFrame(ratioTest443['ratio'].values)


In [206]:

d.insert(1, "src_ip", ratioTest443['ratio'].index, True)

In [207]:
d.columns

Index([0, 'src_ip'], dtype='object')

In [208]:
d.columns[0]

0

In [209]:
d.loc(d[d.columns[0]]>= 1.10*mean443)

TypeError: unhashable type: 'Series'

In [210]:
ax443 = ratioTest53.plot(rot='vertical')
print(mean53)

TypeError: no numeric data to plot

ratioTest53

In [24]:
#Average number of downloaded bytes, per flow, for each source IP
avgUp=pd_test.groupby(['src_ip'])['down_bytes'].mean()
print(avgUp.sort_values())
avgUp.hist?


src_ip
82.155.127.21      91325.659502
82.155.127.131     91966.466038
82.155.127.196     93402.261074
82.155.127.161     93802.233573
82.155.127.63      93896.857479
                      ...      
82.155.127.93      99443.177289
82.155.127.70     100822.137615
82.155.127.33     101208.293578
82.155.127.19     104206.420168
82.155.127.55     104671.643939
Name: down_bytes, Length: 195, dtype: float64


Signature:
avgUp.hist(
    by=None,
    ax=None,
    grid: 'bool' = True,
    xlabelsize: 'int | None' = None,
    xrot: 'float | None' = None,
    ylabelsize: 'int | None' = None,
    yrot: 'float | None' = None,
    figsize: 'tuple[int, int] | None' = None,
    bins: 'int | Sequence[int]' = 10,
    backend: 'str | None' = None,
    legend: 'bool' = False,
    **kwargs,
)
Docstring:
Draw histogram of the input series using matplotlib.

Parameters
----------
by : object, optional
    If passed, then used to form histograms for separate groups.
ax : matplotlib axis object
    If not passed, uses gca().
grid : bool, default True
    Whether to show axis grid lines.
xlabelsize : int, default None
    If specified changes the x-axis label size.
xrot : float, default None
    Rotation of x axis labels.
ylabelsize : int, default None
    If specified changes the y-axis label size.
yrot : float, default None
    Rotation of y axis labels.
figsize : tuple, default None
    Figure size in inche